## Imports

In [2]:
import pandas as pd
import sqlite3

## Read Data

In [3]:
con = sqlite3.connect("../data/nfts.sqlite")
cur = con.cursor()

## List All Available Tables

In [4]:
query = cur.execute("""
SELECT * 
FROM sqlite_master 
WHERE type='table'
""")
cols = [column[0] for column in query.description]
results = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
results

,type,name,tbl_name,rootpage,sql
0,table,nfts,nfts,2,CREATE TABLE nfts\n (\n address TEXT...
1,table,checkpoint,checkpoint,5,CREATE TABLE checkpoint\n (\n event_...
2,table,mints,mints,6,CREATE TABLE mints\n (\n event_id TE...
3,table,transfers,transfers,8,CREATE TABLE transfers\n (\n event_i...
4,table,transfer_values_quartile_10_distribution_per_a...,transfer_values_quartile_10_distribution_per_a...,10,CREATE TABLE transfer_values_quartile_10_distr...
5,table,current_owners,current_owners,11,CREATE TABLE current_owners(\n nft_address TE...
6,table,current_market_values,current_market_values,12,CREATE TABLE current_market_values(\n nft_add...
7,table,market_values_distribution,market_values_distribution,13,CREATE TABLE market_values_distribution(\n ad...
8,table,transfer_statistics_by_address,transfer_statistics_by_address,14,CREATE TABLE transfer_statistics_by_address(\n...
9,table,transfer_values_quantile_10_distribution_per_a...,transfer_values_quantile_10_distribution_per_a...,15,CREATE TABLE transfer_values_quantile_10_distr...


## Union All Transactions

In [6]:
# mints: 6667282
# transfers: 4514729
# combined: 11182011
query = cur.execute("""
SELECT *, 'mint' AS activity_type
FROM mints
""")
cols = [column[0] for column in query.description]
mints_df = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
mints_df.head()

,event_id,transaction_hash,block_number,nft_address,token_id,from_address,to_address,transaction_value,timestamp,activity_type
0,6c6d226e-2096-4b50-ac3f-f7078ca3a0c2,0xe47ac62111db3a6de2957394944501feb7d55cbef00a...,12936415,0x02AA731631c6D7F8241d74F906f5b51724Ab98F8,3223,0x0000000000000000000000000000000000000000,0x8D745Fb3311cC86ED52C177dA0d039C1862F3e13,0.000000e+00,1627776933,mint
1,263c923f-4db9-4d81-a547-27e1fe5ec405,0xe47ac62111db3a6de2957394944501feb7d55cbef00a...,12936415,0x02AA731631c6D7F8241d74F906f5b51724Ab98F8,5077,0x0000000000000000000000000000000000000000,0x8D745Fb3311cC86ED52C177dA0d039C1862F3e13,0.000000e+00,1627776933,mint
2,8baa8502-42f9-453e-824a-ad504045c9c5,0xe47ac62111db3a6de2957394944501feb7d55cbef00a...,12936415,0x02AA731631c6D7F8241d74F906f5b51724Ab98F8,5070,0x0000000000000000000000000000000000000000,0x8D745Fb3311cC86ED52C177dA0d039C1862F3e13,0.000000e+00,1627776933,mint
3,364ee658-ff31-4d0a-a7a7-60d3d7822772,0xe47ac62111db3a6de2957394944501feb7d55cbef00a...,12936415,0x02AA731631c6D7F8241d74F906f5b51724Ab98F8,3757,0x0000000000000000000000000000000000000000,0x8D745Fb3311cC86ED52C177dA0d039C1862F3e13,0.000000e+00,1627776933,mint
4,58d3adc3-03d2-4e7b-b02c-47957c1f5752,0xd44be17c0298717ce001843069b0a2ca08b577fa2b33...,12936414,0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85,3252202771441387673917605047769619801375983664...,0x0000000000000000000000000000000000000000,0x283Af0B28c62C092C9727F1Ee09c02CA627EB7F5,2.251726e+15,1627776925,mint


In [7]:
query = cur.execute("""
SELECT *, 'transfer' as activity_type
FROM transfers
""")
cols = [column[0] for column in query.description]
transfers_df = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
transfers_df.head()

,event_id,transaction_hash,block_number,nft_address,token_id,from_address,to_address,transaction_value,timestamp,activity_type
0,cd816651-56b2-4ed9-887c-c83de732428d,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,6682934142305278177002530107138320246486863356...,0x25f1d709b329C7349b8209851E90eFa3a7f60178,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.0,1627776481,transfer
1,82cc5228-eb80-4e0d-9f6f-e644dec3ab06,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,1198507443711741845273832738859307575893198744...,0x0737E7162C88E9FBB963334e4Bfe6e97447fF811,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.0,1627776481,transfer
2,6e1f9cc4-d1df-4a6b-972d-a20765beb326,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,1541220344281436903042142177888467401468454835...,0xA0fEc186c4b3FAd39ca373dFdd52E79C2495F2c3,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.0,1627776481,transfer
3,43dc27a7-a72d-4894-809c-e868de05f7ee,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,7993840355029527745913866651030234226423689128...,0x90bE22069560A9bAd99543B4A922d1FE34Ae84E9,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.0,1627776481,transfer
4,47b7839b-9b87-442d-b2c1-9ebedcad8e06,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,1123719749142984581497870712442861055686734521...,0xA8E9C42Ebb915EeEC5080B7Bcc5D844475595091,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.0,1627776481,transfer


In [8]:
transactions_df = pd.concat([mints_df, transfers_df])
transactions_df.shape

(11182011, 10)

In [9]:
transactions_df = transactions_df.sort_values(by=['timestamp'], ascending=[True])
transactions_df.head()

,event_id,transaction_hash,block_number,nft_address,token_id,from_address,to_address,transaction_value,timestamp,activity_type
5864146,1b95facf-d43a-4873-a137-dcb552d0a952,0x765644e14b4b24182a59ce9a6f9a22e7b0f5828030c0...,12150245,0x1c903a2F35Fb7c6dC1792dAeC11c033f8B222D5B,19600010002,0x0000000000000000000000000000000000000000,0xE052113bd7D7700d623414a0a4585BCaE754E9d5,0.000000e+00,1617235206,mint
5864147,b88cd541-bd5e-46d4-8eb2-ca2e0e3f2c95,0x85a0f4057a63dfee3e0af93eb53d3788026c542772ce...,12150245,0xbfD1708302137109175907bA0BFba833C43C4c3f,19300030090,0x0000000000000000000000000000000000000000,0xE052113bd7D7700d623414a0a4585BCaE754E9d5,0.000000e+00,1617235206,mint
5864142,7a4c9029-0c6c-4f53-b60f-335fb0ce3ffe,0x40f4312461a1868a169a5dd16e44856e5e778f2a1b5b...,12150246,0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85,2067216718446828862151509780737619171783478638...,0x0000000000000000000000000000000000000000,0x283Af0B28c62C092C9727F1Ee09c02CA627EB7F5,5.749535e+15,1617235214,mint
5864143,3030690a-6e31-496f-9478-c01767486b40,0xf11ff0b3142176bbd2f014cddc3e9abab73a8fce5ea5...,12150246,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,37000182,0x0000000000000000000000000000000000000000,0x34477Ab85120779e938d63A5969bDcEB0C08Bb0d,1.000000e+17,1617235214,mint
5864144,2b14bb65-bff0-409b-bd07-950f1d4c85dd,0x595212d35a1cf0510c4d5f464914952411d1bbc5084b...,12150246,0x3B3ee1931Dc30C1957379FAc9aba94D1C48a5405,17063,0x0000000000000000000000000000000000000000,0x35a031faeE77689544933e1372E2aa75a8DeBe9f,0.000000e+00,1617235214,mint


In [10]:
transactions_df['token_transaction_order'] = transactions_df.groupby(['nft_address','token_id']).cumcount()+1

In [11]:
transactions_df

,event_id,transaction_hash,block_number,nft_address,token_id,from_address,to_address,transaction_value,timestamp,activity_type,token_transaction_order
5864146,1b95facf-d43a-4873-a137-dcb552d0a952,0x765644e14b4b24182a59ce9a6f9a22e7b0f5828030c0...,12150245,0x1c903a2F35Fb7c6dC1792dAeC11c033f8B222D5B,19600010002,0x0000000000000000000000000000000000000000,0xE052113bd7D7700d623414a0a4585BCaE754E9d5,0.000000e+00,1617235206,mint,1
5864147,b88cd541-bd5e-46d4-8eb2-ca2e0e3f2c95,0x85a0f4057a63dfee3e0af93eb53d3788026c542772ce...,12150245,0xbfD1708302137109175907bA0BFba833C43C4c3f,19300030090,0x0000000000000000000000000000000000000000,0xE052113bd7D7700d623414a0a4585BCaE754E9d5,0.000000e+00,1617235206,mint,1
5864142,7a4c9029-0c6c-4f53-b60f-335fb0ce3ffe,0x40f4312461a1868a169a5dd16e44856e5e778f2a1b5b...,12150246,0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85,2067216718446828862151509780737619171783478638...,0x0000000000000000000000000000000000000000,0x283Af0B28c62C092C9727F1Ee09c02CA627EB7F5,5.749535e+15,1617235214,mint,1
5864143,3030690a-6e31-496f-9478-c01767486b40,0xf11ff0b3142176bbd2f014cddc3e9abab73a8fce5ea5...,12150246,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,37000182,0x0000000000000000000000000000000000000000,0x34477Ab85120779e938d63A5969bDcEB0C08Bb0d,1.000000e+17,1617235214,mint,1
5864144,2b14bb65-bff0-409b-bd07-950f1d4c85dd,0x595212d35a1cf0510c4d5f464914952411d1bbc5084b...,12150246,0x3B3ee1931Dc30C1957379FAc9aba94D1C48a5405,17063,0x0000000000000000000000000000000000000000,0x35a031faeE77689544933e1372E2aa75a8DeBe9f,0.000000e+00,1617235214,mint,1
...,...,...,...,...,...,...,...,...,...,...,...
2966262,d43bad91-5ca3-4d53-ac67-07ab03f622a1,0x9ac183f327422c9806b3c8c17551dce1d263a7dd1dc4...,13296011,0xBD9071B63f25Dd199079ED80B3b384D78042956B,929,0x6E0b6f43215C83b57F18F7EB49b492A3ae923278,0xBaf0BDCF25a70c69A49918315583cD0d1D4Be3eD,4.750000e+17,1632586540,transfer,2
2966263,60ba8fb5-a9af-40a0-ac5b-0dc8c18d9c42,0x51dbec47208a1de2024762236251307bb317b941b71b...,13296011,0xb932a70A57673d89f4acfFBE830E8ed7f75Fb9e0,28804,0x55372173689C288552885D897d32f5F706F79aA6,0x830cc132dd66F6491cEAA20206f398247143d9CF,0.000000e+00,1632586540,transfer,2
4456380,06b835cd-b68d-4062-800e-0e5349e92ac9,0x79fa5d33ea6739a8e72ec1f25cb9d6faad15eeb1b5d8...,13296011,0x51Ae5e2533854495f6c587865Af64119db8F59b4,4789,0x0000000000000000000000000000000000000000,0x5ca27a1B8a68233237057C755F14eC186dfD2465,3.000000e+16,1632586540,mint,1
4456381,d30eb7e6-4954-4cc0-a856-2a2475d2e1b5,0x51636307c3976abb028df5238077c7ca59db4215fd2e...,13296011,0xd3B1Ac6D874b8D96f666F25ddDDDd8d52c5A5C83,1186,0x0000000000000000000000000000000000000000,0x7e3d993a8d63c25D1db5F7FF8a8d6C59091003b1,3.000000e+16,1632586540,mint,1


In [14]:
transactions_df[transactions_df["transaction_value"] == 0.0]

,event_id,transaction_hash,block_number,nft_address,token_id,from_address,to_address,transaction_value,timestamp,activity_type
0,6c6d226e-2096-4b50-ac3f-f7078ca3a0c2,0xe47ac62111db3a6de2957394944501feb7d55cbef00a...,12936415,0x02AA731631c6D7F8241d74F906f5b51724Ab98F8,3223,0x0000000000000000000000000000000000000000,0x8D745Fb3311cC86ED52C177dA0d039C1862F3e13,0.0,1627776933,mint
1,263c923f-4db9-4d81-a547-27e1fe5ec405,0xe47ac62111db3a6de2957394944501feb7d55cbef00a...,12936415,0x02AA731631c6D7F8241d74F906f5b51724Ab98F8,5077,0x0000000000000000000000000000000000000000,0x8D745Fb3311cC86ED52C177dA0d039C1862F3e13,0.0,1627776933,mint
2,8baa8502-42f9-453e-824a-ad504045c9c5,0xe47ac62111db3a6de2957394944501feb7d55cbef00a...,12936415,0x02AA731631c6D7F8241d74F906f5b51724Ab98F8,5070,0x0000000000000000000000000000000000000000,0x8D745Fb3311cC86ED52C177dA0d039C1862F3e13,0.0,1627776933,mint
3,364ee658-ff31-4d0a-a7a7-60d3d7822772,0xe47ac62111db3a6de2957394944501feb7d55cbef00a...,12936415,0x02AA731631c6D7F8241d74F906f5b51724Ab98F8,3757,0x0000000000000000000000000000000000000000,0x8D745Fb3311cC86ED52C177dA0d039C1862F3e13,0.0,1627776933,mint
5,86a787d3-f60b-4a60-a197-c4afac308d7a,0x2526fa548f4de7f477d4de51814432f2d3d64c1e5eae...,12936414,0xdf9Aa1012Fa49DC1d2a306e3e65EF1797d2b5fBb,1096,0x0000000000000000000000000000000000000000,0x2448681d0E9D10160f291f90F7DFC1cc67fd9E7B,0.0,1627776925,mint
...,...,...,...,...,...,...,...,...,...,...
4514722,d07bb768-7a9e-4608-a00d-82c1add70173,0x93031fa440039db3e6aa2f11b43acb6a5811f9fb8a50...,12545208,0x3B3ee1931Dc30C1957379FAc9aba94D1C48a5405,43257,0xD4518a94AdC2FCaf1478e7C5CCc029be0f61ced9,0xcDA72070E455bb31C7690a170224Ce43623d0B6f,0.0,1622505415,transfer
4514724,d45af108-4869-44bb-833d-2539faba0f1b,0xed4c976702534f735048e6ffca2a8704904f8aa96b77...,12545210,0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D,5175,0x1b826e30B83A5066474C7E3CA8D9b3880Ba97F09,0x519e19Eac9EAd404343c68224dD2A22C5c25F992,0.0,1622505430,transfer
4514725,73c81c59-c17a-4154-8bbc-5d72a27ee905,0x883257f2403851cf4f1f19a87728aafe269791fe0fcd...,12545211,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,2572313837129053907318259388395403352463424684...,0x327305A797d92a39cEe1a225D7E2A1cC42B1a8fA,0xDe747F2AF4627c1646C206eC7289597039ba319E,0.0,1622505446,transfer
4514726,27f9c904-06cf-4807-9011-d5ecae274249,0xd2643bdc7b6cd192fbdb721b62c8910fbf8753a17605...,12545211,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,2576534060693998439509198862073888456960734364...,0x19804927f74C3c0899AFd49d56bff6ddAE6cF518,0x1b6A054a0850De061763B8C869daF53911e84C91,0.0,1622505446,transfer


In [ ]:
con.close()